In [ ]:
import socket

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)

In [ ]:
import urllib
url = 'https://data.stadt-zuerich.ch/api/3/action/datastore_upsert'  
#fileobj = urllib.urlopen(url)
#print(fileobj.read())



    
response = urllib.request.urlopen(url)
#print(response)
    # convert response to string

string = response.read().decode('utf-8')
#print(string)
    # convert string to json object

json_obj = json.loads(string)
print(json_obj)
    # create a pandas df with the records from the json object

live_data = pd.DataFrame.from_dict(json_obj['result']['records'])
    
json_obj['result']
    
    
    
    
    

In [15]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark
import datetime

df_weather = spark.read\
    .options(header=True, inferSchema=True)\
    .csv('datasets/hourly_weather.csv')

df_accidents = spark

In [18]:
df.show(1)

+---+-------------------+-----------------+-----------------+-------------------+--------------------+--------------------+--------------+------------------+-----------------------+-------------+------------------+----------------+-----------------+-----------+
|_c0|               date|  air_temperature|water_temperature|wind_gust_max_10min|wind_speed_avg_10min|wind_force_avg_10min|wind_direction|         windchill|barometric_pressure_qfe|precipitation|         dew_point|global_radiation|         humidity|water_level|
+---+-------------------+-----------------+-----------------+-------------------+--------------------+--------------------+--------------+------------------+-----------------------+-------------+------------------+----------------+-----------------+-----------+
|  0|2011-01-01 00:30:00|2.233333333333334|              5.2|                2.4|  1.2166666666666668|  1.2166666666666668|          1785|2.2000000000000006|      974.5500000000001|          0.0|1.6166666666666665|

In [20]:
df.dtypes

[('_c0', 'int'),
 ('date', 'string'),
 ('air_temperature', 'double'),
 ('water_temperature', 'double'),
 ('wind_gust_max_10min', 'double'),
 ('wind_speed_avg_10min', 'double'),
 ('wind_force_avg_10min', 'double'),
 ('wind_direction', 'int'),
 ('windchill', 'double'),
 ('barometric_pressure_qfe', 'double'),
 ('precipitation', 'double'),
 ('dew_point', 'double'),
 ('global_radiation', 'double'),
 ('humidity', 'double'),
 ('water_level', 'double')]

In [23]:
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+-------------------+--------------------+--------------------+------------------+------------------+-----------------------+--------------------+------------------+------------------+------------------+-----------------+
|summary|               _c0|               date|   air_temperature| water_temperature|wind_gust_max_10min|wind_speed_avg_10min|wind_force_avg_10min|    wind_direction|         windchill|barometric_pressure_qfe|       precipitation|         dew_point|  global_radiation|          humidity|      water_level|
+-------+------------------+-------------------+------------------+------------------+-------------------+--------------------+--------------------+------------------+------------------+-----------------------+--------------------+------------------+------------------+------------------+-----------------+
|  count|             78707|              78707|             78707|            

In [10]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark
import datetime

spark = SparkSession.builder.getOrCreate()

In [11]:
df = spark.read.csv('datasets/hourly_weather.csv', header=True, inferSchema=True)

In [12]:
df.columns

['_c0',
 'date',
 'air_temperature',
 'water_temperature',
 'wind_gust_max_10min',
 'wind_speed_avg_10min',
 'wind_force_avg_10min',
 'wind_direction',
 'windchill',
 'barometric_pressure_qfe',
 'precipitation',
 'dew_point',
 'global_radiation',
 'humidity',
 'water_level']

In [13]:
df.groupBy('date').count().show(10)

+-------------------+-----+
|               date|count|
+-------------------+-----+
|2011-01-01 04:30:00|    1|
|2011-01-02 00:30:00|    1|
|2011-02-08 07:30:00|    1|
|2011-02-13 14:30:00|    1|
|2011-02-18 20:30:00|    1|
|2011-02-26 15:30:00|    1|
|2011-03-04 06:30:00|    1|
|2011-03-14 03:30:00|    1|
|2011-03-19 04:30:00|    1|
|2011-03-24 02:30:00|    1|
+-------------------+-----+
only showing top 10 rows



In [ ]:
%%time
steps = df.select('date').distinct().collect()
for step in steps[:]:
    _df = df
    _df.coalesce(1).write.mode("append").option("header", "true").csv("logs/")

In [ ]:
!cd logs/ && ls

In [ ]:
part = spark.read.csv(
    'logs/part-00000-002e8f2f-8449-4795-ad3c-f97a929cc814-c000.csv',
    header=True,
    inferSchema=True,

)

In [ ]:
part.groupBy("date").count().show()

In [ ]:
dataSchema = part.schema

In [ ]:
dataSchema

In [ ]:
streaming = (
    spark.readStream.schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .csv("logs/")
)

In [ ]:
dest_count = streaming
dest_count

In [ ]:
activityQuery = {
    dest_count.writeStream.queryName("dest_counts")
    .format("memory")
    .outputMode('complete')
    .start()
}

import time

for x in range(50):
    _df = spark.sql(
        'SELECT * FROM dest_count'
    )
    if _df.count() > 1:
        _df.show(10)
    time.sleep(1)

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 5)

In [3]:
# Create a DStream that will connect to hostname:port, like localhost:9999
lines = sc.textFile('http://localhost:9870/explorer.html#/weather/hourly_weather.csv', 9870)

In [5]:
print(lines)

http://localhost:9870/explorer.html#/weather/hourly_weather.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [ ]:
# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

In [ ]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [ ]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

In [ ]:
import urllib.request
import json
import pandas as pd
import threading
from datetime import datetime

In [ ]:
data_url = 'https://data.stadt-zuerich.ch/api/3/action/datastore_search?resource_id=ab29e6fa-1e79-441a-b096-37c23cf3d2be&limit=5'

def getLiveData(url):

    response = urllib.request.urlopen(url)

    # convert response to string

    string = response.read().decode('utf-8')

    # convert string to json object

    json_obj = json.loads(string)

    # create a pandas df with the records from the json object

    live_data = pd.DataFrame.from_dict(json_obj['result']['records'])
    return live_data

def writeLogs():
    
    # repeat function every 10 seconds
    
    threading.Timer(10, writeLogs).start()
    
    # pull data and write log file
    
    now = datetime.now()
    getLiveData(data_url).to_csv('logs/log' + str(now.strftime("%m%d%Y;%H:%M:%S")))

In [ ]:
# now = datetime.now()
# getLiveData(data_url).to_csv('logs/log' + str(now.strftime("%m%d%Y;%H:%M:%S")))
print(getLiveData(data_url))

In [ ]:
from datetime import datetime

now = datetime.now()
str(now.strftime("%m%d%Y;%H:%M:%S"))

In [ ]:
test = pd.read_csv('logs/log05202021;14:03:16')

In [ ]:
writeLogs()

In [ ]:
from pyspark.streaming import StreamingContext
from pyspark import SparkConf

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

stream = sc.textFileStream('logs/')

In [ ]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
ssc = StreamingContext(sc, 5)
data = ssc.textFile("/home/bigdata/test.txt")

In [ ]:


response = urllib.request.urlopen(data_url)

# convert response to string

string = response.read().decode('utf-8')

# convert string to json object

json_obj = json.loads(string)

# create a pandas df with the records from the json object

live_data = pd.DataFrame.from_dict(json_obj['result']['records'])
live_data